<h1><center>Run spatial queries in PySpark</center></h1>


This notebook shows you show to use use spatial queries in Spark environments. The notebook uses the spatio-temporal library that is pre-installed on all Spark environments in Watson Studio. You will learn how to perform common spatial queries in Spark. 

The types of spatial queries you will learn to use are:
- In a set of points, find all the points that are within a certain distance to a particular point. For example, find all the hospitals that are within a certain distance to a given location.
- In a set of polygons, find all the polygons that contain a particular point. For example, find all the risk areas for fires, floods, or hurricanes that contain a particular location.
- In a set of points, find all the points that are contained within a particular polygon. For example, find all the retail outlets in a particular region.

Often, a spatial function has one parameter that refers to a spatial column in one table and a second parameter that refers to a spatial constant or to a spatial column in another table. This notebook shows you how to use functions to access and combine data of different types to perform spatial queries.

This notebook runs on Python 3.6 with Spark.


## Table of Contents


1.	[Register the Spark SQL spatial functions](#register)
2.	[Get sample data](#getData)
3.	[Create a geometry column](#createColumn)
4.	[Register the data frames](#registerDataframe)
5.  [Run spatial queries](#runQueries)  
6.	[Summary](#summary)




<a id="register"></a>
## 1. Register the Spark SQL spatial functions

Register the Spark SQL spatial functions:

In [1]:
spark._jvm.org.apache.spark.sql.types.SqlGeometry.registerAll(spark._jsparkSession)

NameError: name 'spark' is not defined

<a id="getData"></a>
## 2. Get sample data

This notebook uses a sample data set that is available in the IBM Watson Studio Gallery.

1. Click [<name_of_data_set>](<link_to_the_data_set))to access the data set in the Watson Studio Gallery.
2. Click **Add to project** and select your project. 
   The data file will be added to the selected project's Assets page as a data asset. 
3. Click the Find and add data icon from this notebook's action bar. On the files tab, you will see the sample file you added to the project.
4. To load the data from the file into a data frame, click in the next code cell and select **Insert to code > Insert SparkSession  DataFrame** under the file name.

Note: The data frame that is created for you and filled with data is given a generic name (`df_data_1`). Rename the data frame to `hospital_df`  and run the code cell to proceed.

Read the hospital data where each hospital's location is a latitude-longitude point:

In [2]:
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-e4954516-f3e2-4418-aca0-ee9ee4bebca0',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token',
    'api_key': 'mUL3zgMRm9hpmwclEPyYiEa_qbtA1aijcdjjZGSquEvY'
}

configuration_name = 'os_79ac2ad149064ac5a2fb7c7fc32ada05_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

hospital_df = spark.read.csv(cos.url('hospitals.csv', 'spatiotemporalpythonexamples-donotdelete-pr-1ph9s1kbmrodxt'), header=True)

hospital_df.show(3, False)

+---+-----------------------------+------------+-----+----------+---------+
|id |name                         |city        |state|lon       |lat      |
+---+-----------------------------+------------+-----+----------+---------+
|1  |Southern Hills Medical Center|BERRY HILL  |TN   |-86.721939|36.077843|
|2  |Sycamore Shoals Hospital     |ELIZABETHTON|TN   |-82.247635|36.346218|
|3  |Tokona Hospital              |GREENEVILLE |TN   |-82.845711|36.151772|
+---+-----------------------------+------------+-----+----------+---------+
only showing top 3 rows



Read the county data where each county is a polygon/multipolygon:

In [3]:
county_df = spark.read.csv(cos.url('counties_WKT_with_header.csv', 'spatiotemporalpythonexamples-donotdelete-pr-1ph9s1kbmrodxt'), header=True)
county_df.select('name', 'shape_wkt').show(3)

+-----------------+--------------------+
|             name|           shape_wkt|
+-----------------+--------------------+
|Lake of the Woods|MULTIPOLYGON (((-...|
|            Ferry|MULTIPOLYGON (((-...|
|          Stevens|MULTIPOLYGON (((-...|
+-----------------+--------------------+
only showing top 3 rows



<a id="createColumn"></a>
## 3. Create a geometry column for hospital and county data

The raw spatial data in the data frame can be of various types, for example **columns indicating lat and lon** or **column indicating wkt string for the geometry**, and so on.

Therefore, the first step is to use a spatial query to generate a new spatial column that combines the data in these columns.  
For example, use the function:
- `ST_Point(lon_col, lat_col)` if the raw spatial data is in a latitude column and a longitude column  
- `ST_WKTTOSQL(wkt_col)` if the raw spatial data is in a column containing the wkt string form of the geometry  

For the full list of possible query functions, see [Geospatial Toolkit functions](https://www.ibm.com/support/knowledgecenter/en/SSCJDQ/com.ibm.swg.im.dashdb.analytics.doc/doc/geo_functions.html).

Create a geometry column for the hospital data using `ST_Point(lon, lat)`:

In [4]:
hospital_df.createOrReplaceTempView("hospitals")
hospital_df = spark.sql("SELECT *, ST_Point(lon, lat) as location from hospitals")
hospital_df.show(3, False)

+---+-----------------------------+------------+-----+----------+---------+---------------------------------------+
|id |name                         |city        |state|lon       |lat      |location                               |
+---+-----------------------------+------------+-----+----------+---------+---------------------------------------+
|1  |Southern Hills Medical Center|BERRY HILL  |TN   |-86.721939|36.077843|PointEG: lat=36.077843, long=-86.721939|
|2  |Sycamore Shoals Hospital     |ELIZABETHTON|TN   |-82.247635|36.346218|PointEG: lat=36.346218, long=-82.247635|
|3  |Tokona Hospital              |GREENEVILLE |TN   |-82.845711|36.151772|PointEG: lat=36.151772, long=-82.845711|
+---+-----------------------------+------------+-----+----------+---------+---------------------------------------+
only showing top 3 rows



Create a geometry column for the county data using `ST_WKTToSQL(wkt_string)`:

In [5]:
county_df.createOrReplaceTempView('counties')
county_df = spark.sql("SELECT NAME, STATE_NAME, POP2000, ST_WKTToSQL(shape_WKT) as shape from counties")
county_df.show(3)

+-----------------+----------+-------+--------------------+
|             NAME|STATE_NAME|POP2000|               shape|
+-----------------+----------+-------+--------------------+
|Lake of the Woods| Minnesota|   4522|AcceleratedMultiP...|
|            Ferry|Washington|   7260|AcceleratedMultiP...|
|          Stevens|Washington|  40066|AcceleratedMultiP...|
+-----------------+----------+-------+--------------------+
only showing top 3 rows



<a id="registerDataframe"></a>
## 4. Register the hospital and county data frames as a temporary view

A data frame can also be used to create a temporary view. Registering a data frame as a table allows you to run SQL queries over its data. Register the hospital and county data frames as a temporary view: 

In [6]:
hospital_df.createOrReplaceTempView('hospitals')
county_df.createOrReplaceTempView('counties')

<a id="runQueries"></a>
## 5. Run spatial queries

### Example 1: Query to determine points closest to another point

This sample query shows you how to find the hospitals that are within a certain distance of a given location (which is constructed using the ST_Point constructor).

In [7]:
spark.sql("""
SELECT name, city, state
FROM hospitals
WHERE ST_Distance(location, ST_Point(-77.574722, 43.146732)) < 10000.0
""").show()

+--------------------+-----------+-----+
|                name|       city|state|
+--------------------+-----------+-----+
|Park Avenue Hospital|   BRIGHTON|   NY|
|County Home and I...|   BRIGHTON|   NY|
|    Genesee Hospital|   BRIGHTON|   NY|
|   Highland Hospital|   BRIGHTON|   NY|
|Rochester General...|IRONDEQUOIT|   NY|
|Saint Marys Hospital|   BRIGHTON|   NY|
|Strong Memorial H...|   BRIGHTON|   NY|
+--------------------+-----------+-----+



### Example 2: Queries to determine which polygon contains a point

The following sample queries show you how to use spatial functions to determine which polygon contains a given point. The examples use the following functions:

1. `ST_Contains(geom1, geom2)`: This function returns TRUE if the `geom2` values are completely contained by the polygons identified by `geom1`.
2. `ST_Within(geom1, geom2)`: This function returns TRUE if the `geom1` values are within the polygons identified by `geom2`.
3. `ST_Intersects(geom1, geom2)`: This function returns TRUE if `geom1` and `geom2` intersect spatially in any way. This can be that they  touch, cross, or contain each other.

In [ ]:
spark.sql("""
SELECT NAME 
FROM counties 
WHERE ST_Contains(shape, ST_Point(-74.237, 42.037))
""").show()

In [9]:
spark.sql("""
SELECT NAME
FROM counties
WHERE
ST_Within(ST_Point(-74.237, 42.037), shape)
""").show()

+------+
|  NAME|
+------+
|Ulster|
+------+



In [10]:
spark.sql("""
SELECT NAME
FROM counties
WHERE
ST_Intersects(shape, ST_Point(-74.237, 42.037))
""").show()

+------+
|  NAME|
+------+
|Ulster|
+------+



### Example 3: Queries to determine the points in a polygon

Each of the following queries determines which hospitals are located within the specified polygon, which is defined as a constant using the  well-known text (WKT) representation. The polygon definition consists of the character string POLYGON followed by a pair of x,y coordinates for each vertex, separated by a comma. The individual x and y values are separated by a space. The entire list of coordinate pairs must be in parentheses.

In [11]:
spark.sql("""
SELECT name
FROM hospitals
WHERE
ST_Contains(ST_WKTToSQL('POLYGON ((-74.0 42.0, -73.0 42.0, -73.0 43.0, -74.0 43.0, -74.0 42.0))'), location)
""").show(3)

+--------------------+
|                name|
+--------------------+
|   Marshall Hospital|
|Southwestern Medi...|
|  Hillcrest Hospital|
+--------------------+
only showing top 3 rows



In [12]:
spark.sql("""
SELECT name 
FROM hospitals 
WHERE ST_Within(location, ST_WKTToSQL('POLYGON ((-74.0 42.0, -73.0 42.0, -73.0 43.0, -74.0 43.0, -74.0 42.0))'))
""").show(3)

+--------------------+
|                name|
+--------------------+
|   Marshall Hospital|
|Southwestern Medi...|
|  Hillcrest Hospital|
+--------------------+
only showing top 3 rows



In [13]:
spark.sql("""
SELECT name 
FROM hospitals 
WHERE ST_Intersects(location, ST_WKTToSQL('POLYGON ((-74.0 42.0, -73.0 42.0, -73.0 43.0, -74.0 43.0, -74.0 42.0))'))
""").show(3)

+--------------------+
|                name|
+--------------------+
|   Marshall Hospital|
|Southwestern Medi...|
|  Hillcrest Hospital|
+--------------------+
only showing top 3 rows



### Example 4: Spatial join queries to determine points in a polygon

Just as a regular join function can join two tables based on the values in columns that contain character or numeric data, spatial join functions can be used to join tables based on the values in the columns that contain spatial data. The following examples use the **counties** and **hospitals** tables.

You can use the spatial join function to find the hospitals located within a specific county. For example, the following query returns a list of all the hospitals in Dutchess county:

In [14]:
spark.sql("""
SELECT c.NAME, h.name 
FROM counties AS c, hospitals AS h 
WHERE c.NAME = 'Dutchess' 
AND ST_Intersects(c.shape, h.location)
""").show()

+--------+--------------------+
|    NAME|                name|
+--------+--------------------+
|Dutchess|Hudson River Stat...|
|Dutchess|Vassar Brothers H...|
|Dutchess|      Bowne Hospital|
|Dutchess|Harlem Valley Sta...|
|Dutchess|Matteawan State H...|
|Dutchess|New York State Ho...|
|Dutchess|Saint Francis Hos...|
|Dutchess|United States Vet...|
+--------+--------------------+



Alternatively, you can use the SQL `JOIN ... ON ...` notation, which is equivalent to a spatial predicate in the WHERE clause. For example, the following query produces the same result set as the previous query:

In [15]:
spark.sql("""
SELECT h.name, c.NAME
FROM counties AS c
JOIN hospitals AS h
ON c.NAME = 'Dutchess'
AND ST_Intersects(h.location, c.shape)
""").show()

+--------------------+--------+
|                name|    NAME|
+--------------------+--------+
|Hudson River Stat...|Dutchess|
|Vassar Brothers H...|Dutchess|
|      Bowne Hospital|Dutchess|
|Harlem Valley Sta...|Dutchess|
|Matteawan State H...|Dutchess|
|New York State Ho...|Dutchess|
|Saint Francis Hos...|Dutchess|
|United States Vet...|Dutchess|
+--------------------+--------+



The following query returns the name of the county in which a particular hospital is located:

In [16]:
spark.sql("""
SELECT c.NAME, h.name
FROM hospitals AS h, counties AS c
WHERE ST_Intersects(h.location, c.shape)
AND h.name = 'Vassar Brothers Hospital'
""").show()

+--------+--------------------+
|    NAME|                name|
+--------+--------------------+
|Dutchess|Vassar Brothers H...|
+--------+--------------------+



### Example 5: Spatial join queries with additional predicates and aggregation

This example shows you how to use spatial joins in conjunction with additional predicates and aggregation, which can address business problems. These examples continue to use the hospitals and counties tables, but the same principles could be applied to any other type of data.

The following example queries the hospitals within each county in New York state, qualifying by the state name in the counties table.

In [17]:
spark.sql("""
SELECT c.NAME, h.name
FROM counties AS c, hospitals AS h
WHERE ST_Intersects(h.location, c.shape)
AND c.STATE_NAME='New York'
ORDER BY c.NAME, h.name
""").show(3)

+------+--------------------+
|  NAME|                name|
+------+--------------------+
|Albany|     Albany Hospital|
|Albany|Albany Hospital F...|
|Albany|Albany Hospital S...|
+------+--------------------+
only showing top 3 rows



The same results can be obtained by rewriting the above query and using the field from the hospitals table:

In [18]:
spark.sql("""
SELECT c.NAME, h.name
FROM hospitals AS h, counties AS c
WHERE ST_Intersects(h.location, c.shape)
AND h.state='NY'
ORDER BY c.NAME, h.name
""").show(3)

+------+--------------------+
|  NAME|                name|
+------+--------------------+
|Albany|     Albany Hospital|
|Albany|Albany Hospital F...|
|Albany|Albany Hospital S...|
+------+--------------------+
only showing top 3 rows



The following example lists the number of hospitals per county in New York:

In [19]:
spark.sql("""
SELECT c.NAME, COUNT(h.name) AS hospital_count
FROM counties AS c, hospitals AS h
WHERE ST_Intersects(h.location, c.shape)
AND c.STATE_NAME='New York'
GROUP BY c.NAME
""").show(3)

+------+--------------+
|  NAME|hospital_count|
+------+--------------+
|Cayuga|             1|
| Kings|            26|
|Monroe|             9|
+------+--------------+
only showing top 3 rows



To identify counties where the population is underserved by hospitals, an interesting metric might be the number of people per hospital in each county. Using the population of each county in the year 2000, you can calculate this number.

In [20]:
spark.sql("""
SELECT c.NAME, 
COUNT(h.name) AS hospital_count, 
c.POP2000 AS Population, 
c.POP2000/COUNT(h.name) AS people_per_hospital
FROM counties AS c, hospitals AS h
WHERE c.STATE_NAME='New York'
AND ST_Intersects(h.location, c.shape)
GROUP BY c.NAME, c.POP2000
ORDER BY people_per_hospital DESC
""").show(3)

+----------+--------------+----------+-------------------+
|      NAME|hospital_count|Population|people_per_hospital|
+----------+--------------+----------+-------------------+
|     Bronx|             9|   1332650| 148072.22222222222|
|Chautauqua|             1|    139750|           139750.0|
|    Oswego|             1|    122377|           122377.0|
+----------+--------------+----------+-------------------+
only showing top 3 rows



With additional detail, such as number of beds, number of doctors per hospital, you could determine a better measure for health care coverage per state and population.

### Example 6: Window queries

A common use case for mapping applications and in particular for web mapping is to select objects that fall within a rectangular region. This can be done by creating a polygon to represent the rectangle and using the `ST_Intersects` spatial predicate.

In [21]:
spark.sql("""
SELECT name
FROM hospitals
WHERE ST_Intersects(location, ST_WKTToSQL(
 'POLYGON ((-74.0 42.0, -73.0 42.0, -73.0 43.0, -74.0 43.0, -74.0 42.0))'))
""").show(3)

+--------------------+
|                name|
+--------------------+
|   Marshall Hospital|
|Southwestern Medi...|
|  Hillcrest Hospital|
+--------------------+
only showing top 3 rows



Another spatial predicate that does the same is `EnvelopesIntersect`, which can be used to select objects whose envelope intersects a rectangular region. `EnvelopesIntersect` takes as a parameter the name of the spatial column and four Double values representing the lower-left, and upper-right corners of the rectangle. This spatial predicate is simpler to use and is more efficient than `ST_Intersects` for rectangular windows.

In [22]:
spark.sql("""
SELECT name
FROM hospitals
WHERE EnvelopesIntersect(location, -74.0, 42.0, -73.0, 43.0)
""").show(3)

+--------------------+
|                name|
+--------------------+
|   Marshall Hospital|
|Southwestern Medi...|
|  Hillcrest Hospital|
+--------------------+
only showing top 3 rows



Because this predicate is true if any portion of a line or polygon geometry falls within the specified window, parts of the line or polygon might lie outside of the window. This is generally not a problem with mapping applications, which will discard geometries that lie outside the display window.

When building web-mapping applications with widely used web-mapping APIs from providers such as Google Maps, Yahoo! Maps, Bing Maps, and others, it is necessary to provide the longitude and latitude values to be used in placing custom markers on the map. You can get this information by using a query such as the following:

In [23]:
spark.sql("""
SELECT name, ST_X(location) AS longitude, ST_Y(location) AS latitude
FROM hospitals
WHERE EnvelopesIntersect(location, -74.0, 42.0, -73.0, 43.0)
""").show(3)

+--------------------+----------+---------+
|                name| longitude| latitude|
+--------------------+----------+---------+
|   Marshall Hospital|-73.678177|42.718971|
|Southwestern Medi...|-73.206772|42.874249|
|  Hillcrest Hospital|-73.280113|42.457863|
+--------------------+----------+---------+
only showing top 3 rows



### Example 7: Distance queries

Another common spatial query is to find things within a specified distance of a particular location. You have probably used web-mapping applications to get this kind of information. You can issue SQL queries from your application for questions like:

- Find customers within 10 miles of a store
- Find ATMs within 500 meters of the current location
- Find competitive stores within 10 kilometers of a proposed store location

The spatial function used for these queries is `ST_Distance`, which computes the distance between the spatial values and returns a result in meters. 

The following query generates eight results:

In [24]:
spark.sql("""
SELECT name
FROM hospitals
WHERE ST_Intersects(location, ST_WKTToSQL(
 'POLYGON ((-74.0 42.0, -73.0 42.0, -73.0 43.0, -74.0 43.0, -74.0 42.0))'))
""").show(3)

+--------------------+
|                name|
+--------------------+
|   Marshall Hospital|
|Southwestern Medi...|
|  Hillcrest Hospital|
+--------------------+
only showing top 3 rows



A different way of querying the same location above is to use the `ST_Buffer` function, where a circular buffer is created around the given geometry and the desired geometries within that buffer are determined. The `ST_Buffer` function takes as parameters a spatial geometry and a distance in meters to the buffer around this spatial value. The results are the same as when you us `ST_Intersects`.

In [25]:
spark.sql("""
SELECT name
FROM hospitals
WHERE
ST_Intersects(location,
  ST_Buffer(ST_Point(-74.237, 42.037), 46800.0))
ORDER BY name
""").show(3)

+--------------------+
|                name|
+--------------------+
|      Adventist Home|
|      Bowne Hospital|
|Columbia Memorial...|
+--------------------+
only showing top 3 rows



The following query returns the distance from a specified point to each object within a 30 mile (or approximately 46800m) radius:

In [26]:
spark.sql("""
SELECT name, ST_Distance(location, ST_Point(-74.237, 42.037)) AS distance
FROM hospitals
WHERE ST_Distance(location, ST_Point(-74.237, 42.037)) < 46800.0
ORDER BY distance
""").show(3)

+--------------------+-----------------+
|                name|         distance|
+--------------------+-----------------+
|Greene County Mem...|36634.88406671601|
|      Adventist Home|39014.09079207157|
|Hudson River Stat...|43362.54508885234|
+--------------------+-----------------+
only showing top 3 rows



You could also use `ST_Buffer` to compute the spatial relation and then determine the distance as is shown in the following query:

In [27]:
spark.sql("""
SELECT name, ST_Distance(location, ST_Point(-74.237, 42.037)) AS distance
FROM hospitals
WHERE
  ST_Intersects(location,
  ST_Buffer(ST_Point(-74.237, 42.037), 46800.0))
ORDER BY distance
""").show(3)

+--------------------+-----------------+
|                name|         distance|
+--------------------+-----------------+
|Greene County Mem...|36634.88406671601|
|      Adventist Home|39014.09079207157|
|Hudson River Stat...|43362.54508885234|
+--------------------+-----------------+
only showing top 3 rows



A key difference that you should note here is that the `ST_Buffer` function in the spatio-temporal library supports buffering of arbitrary geometries and can be used to compute in that manner. 

Bear in mind that:
- The ST_Buffer query on large geometries can be expensive.
- For a large number of geometries, you are advised to calculate the buffers separately, to store the buffers in columns, and to then operate on the stored buffers.

In [28]:
spark.sql("""
SELECT name, ST_Distance(location, ST_WKTToSQL(
 'LINESTRING (-74.0 42.0, -73.0 42.0)'))
FROM hospitals
WHERE ST_Intersects(location, ST_Buffer(ST_WKTToSQL(
 'LINESTRING (-74.0 42.0, -73.0 42.0)'), 46800.0))
""").show(3)

+--------------------+-------------------------------------------------------------------------------+
|                name|UDF:ST_Distance(location, UDF:ST_WKTToSQL(LINESTRING (-74.0 42.0, -73.0 42.0)))|
+--------------------+-------------------------------------------------------------------------------+
|      Avery Hospital|                                                             38777.964957148106|
|   Hartford Hospital|                                                               38204.0148377887|
|Springfield Munic...|                                                              39761.18673983218|
+--------------------+-------------------------------------------------------------------------------+
only showing top 3 rows



<a id="summary"></a>
##  Summary

In this notebook, you learnt how to query spatial data you downloaded from the IBM Watson Studio Gallery. You registered each data frame (one with data on hospitals and another with county information) as a table to run your queries on. The sample queries showed you how to determine the hospitals within a certain distance or in a polygon, to find the name of the county in which a hospital is located, or to identify the counties where the population is underserved by hospitals. The sample queries showed you how to use and combine the most common Spark SQL spatial functions in queries. 